In [1]:
import os
import json
import re
import glob

import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns

import plotly
import plotly.express as px
plotly.__version__

'4.14.3'

---

# **Global variables**

In [4]:
# defines where files are located
root = '/hpc/brainets/data/db_ebrains/seeg/Cohort %s/%s/%s/*_MCSE_*.pos'
subject = 'HID-Sub-000'
sfreq = 64.

# load the cohort config file
with open("../../seeg-ebrains/config/db_cohort.json", 'r') as f:
    cfg = json.load(f)

---

# **Global functions**
## Read the triggers

In [5]:
def read_pos(path_pos):
    """Path to the pos file and return a dataframe containing the triggers and samples
    at which triggers appeared."""
    # read the file
    r = np.genfromtxt(path_pos)
    assert (r.ndim == 2) and (r.shape[1] == 3)
    # dataframe conversion
    df = pd.DataFrame(r[:, 0:2].astype(int), columns=['sample', 'trigger'])
    # trigger remapping
    remap = {11: 10, 12: 10, 61: 60, 62: 60, 1: 100, 2: 200}
    return df

## Detect events

In [6]:
def detect_events(df):
    """Detect events.
    
    This function takes as input the dataframe containing triggers and samples
    and return a dataframe which all of the detected events (easy /hard blocks;
    reponse; response time; correct / incorrect)
    """
    # detect consecutive events
    block, response, t_block = [], [], None
    for n_t, t in enumerate(list(df['trigger'])):
        if t in [11, 12, 61, 62]:
            t_block = n_t
        elif t in [1, 2] and t_block is not None:
            block += [t_block]
            response += [n_t]
            t_block = None
    # build dataframe
    dat = np.c_[df.loc[block, 'sample'], df.loc[response, 'sample'],
                df.loc[block, 'trigger'], df.loc[response, 'trigger']].astype(int)
    df_t = pd.DataFrame(dat, columns=['sample_stim', 'sample_resp', 'trig_stim', 'trig_resp'])
    # insert timings
    df_t['time_stim'] = df_t['sample_stim'] / sfreq
    df_t['time_resp'] = df_t['sample_resp'] / sfreq
    df_t['response_time'] = df_t['time_resp'] - df_t['time_stim']
    # add if correct answer
    df_t['correct'] = np.mod(dat[:, -2] - dat[:, -1], 10) == 0
    # add column for difficulty of the task
    df_t['difficulty'] = ['easy'] * len(df_t)
    stim = np.array(list(df_t['trig_stim']))
    df_t.loc[np.logical_or(stim == 61, stim == 62), 'difficulty'] = 'hard'
    # add orientation of the "T" letter
    df_t['T_orientation'] = ['T down'] * len(df_t)
    df_t.loc[np.logical_or(stim == 12, stim == 62), 'T_orientation'] = 'T up'
    return df_t


---

# **Detect and save the events**

In [11]:
##########################################################################
save_to = '/hpc/brainets/data/db_ebrains/analysis/events'
##########################################################################

if not os.path.isdir(save_to): os.mkdir(save_to)
    
for coh in ['I', 'II', 'III']:
    suj_range = cfg[coh]['s_range']
    seeg_folder = cfg[coh]['seeg_folder']
    for suj in range(suj_range[0], suj_range[1] + 1):
        print(f"Cohort {coh} : {suj}", end='\r')
        # define subject name
        id_suj = str(suj)
        s_name = subject[0:-len(id_suj)] + id_suj
        save_as = os.path.join(save_to, f"{s_name}.xlsx")
        
        # if already saved, skip
        if os.path.isfile(save_as):
            print(f" -----------SUBJECT {suj} ALREADY EXTRACTED. SKIPPED -----------")
            continue
        
        # define path to the pos file
        path_pos = root % (coh, seeg_folder, s_name)
        
        # get pos file name
        file = glob.glob(path_pos)
        if len(file) != 1:
            print(f"Subject {suj} has no file. SKIPPED")
            continue
        
        # read the triggers and events
        df_trig = read_pos(file[0])
        df_ev = detect_events(df_trig)
        
        # track those fow which events detection failed
        if len(df_ev) < 10:
            # here we say that if there's not at least, 10 events, it's considered
            # as a bad file
            print(f"SUBJECT {suj} FAILED (not enough trials. SKIPPED)")
            continue

        # save the events
        
        df_ev.to_excel(save_as)

 -----------SUBJECT 1 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 2 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 3 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 4 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 5 ALREADY EXTRACTED. SKIPPED -----------
SUBJECT 6 FAILED (not enough trials. SKIPPED)
SUBJECT 7 FAILED (not enough trials. SKIPPED)
 -----------SUBJECT 8 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 9 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 10 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 11 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 12 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 13 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 14 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 15 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 16 ALREADY EXTRACTED. SKIPPED -----------
 -----------SUBJECT 17 ALREADY EXTRACTED. SKIPP